## 1、FP树：用于编码数据集的有效方式

> 优点：一般要快于Apriori

> 缺点：实现比较困难，在某些数据集上性能会下降

> 适用数据类型：标称型数据

FP-growth算法的工作流程如下，首先构建FP树，然后利用它来挖掘频繁项集。

为构建FP树，需要对原始数据集扫描两遍，第一遍对所有元素项的出现次数进行技术，第二遍扫描中只考虑那些频繁元素。

## 2、构建FP树

### 2.1 创建FP树的数据结构

In [1]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
        
    def inc(self, numOccur):
        self.count += numOccur
    
    def disp(self, ind=1):
        print(' '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [2]:
rootNode = treeNode('pyramid', 9, None)

# 增加一个节点
rootNode.children['eye'] = treeNode('eye', 13, None)
rootNode.disp()

  pyramid   9
   eye   13


In [3]:
rootNode.children['phoenix'] = treeNode('phoenix', 3, None)
rootNode.disp()

  pyramid   9
   eye   13
   phoenix   3


### 2.2 构建FP树

In [19]:
def createTree(dataSet, minSup=1):
    headerTable = {}
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in list(headerTable.keys()):
        if headerTable[k] < minSup:
            del headerTable[k]
    freqItemSet = set(headerTable.keys())
    if len(freqItemSet) == 0:
        return None, None
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]
    retTree = treeNode('Null Set', 1, None)
    for tranSet, count in dataSet.items():
        localD = {}
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)
    return retTree, headerTable

In [6]:
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:
        inTree.children[items[0]].inc(count)
    else:
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    
    if len(items) > 1:
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

In [7]:
def updateHeader(nodeToTest, targetNode):
    while nodeToTest.nodeLink != None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [8]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

In [9]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [10]:
simpDat = loadSimpDat()
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [11]:
initSet = createInitSet(simpDat)
initSet

{frozenset({'z'}): 1,
 frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [20]:
myFPtree, myHeaderTab = createTree(initSet, 3)
myFPtree.disp()

  Null Set   1
   z   5
    r   1
    x   3
     y   3
      s   2
       t   2
      t   1
       r   1
   x   1
    s   1
     r   1


已经构建了FP树，接下来就使用它进行频繁项集挖掘。

## 3、从一棵FP树中挖掘频繁项集

从FP树中抽取频繁项集的三个基本步骤是：

1.从FP树中获得条件模式基

2.利用条件模式基，构建一个条件FP树

3.迭代重复步骤（1）和（2），直到树包含一个元素项为止

### 3.1 抽取条件模式基（conditional pattern base）

In [22]:
# 迭代上溯整棵树
def ascendTree(leafNode, prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

In [23]:
def findPrefixPath(basePat, treeNode):
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [26]:
findPrefixPath('x', myHeaderTab['x'][1])

{frozenset({'z'}): 3}

### 3.2 创建条件FP树

In [44]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[0])]
    
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        myCondTree, myHead = createTree(condPattBases, minSup)
        #print('conditional tree for: {0}'.format(newFreqSet))
        if myCondTree != None:
            myCondTree.disp()
        if myHead != None:
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [39]:
freqItems = []
mineTree(myFPtree, myHeaderTab, 3, set([]), freqItems)

conditional tree for: {'r'}
conditional tree for: {'s'}
  Null Set   1
   x   3
conditional tree for: {'x', 's'}
conditional tree for: {'t'}
  Null Set   1
   z   2
    y   2
     x   2
   y   1
    x   1
     z   1
conditional tree for: {'t', 'x'}
  Null Set   1
   y   3
conditional tree for: {'t', 'x', 'y'}
conditional tree for: {'t', 'y'}
conditional tree for: {'t', 'z'}
conditional tree for: {'x'}
  Null Set   1
   z   3
conditional tree for: {'x', 'z'}
conditional tree for: {'y'}
  Null Set   1
   x   3
    z   3
conditional tree for: {'y', 'x'}
conditional tree for: {'y', 'z'}
  Null Set   1
   x   3
conditional tree for: {'x', 'y', 'z'}
conditional tree for: {'z'}


In [37]:
freqItems

[{'r'},
 {'s'},
 {'s', 'x'},
 {'t'},
 {'t', 'x'},
 {'t', 'x', 'y'},
 {'t', 'y'},
 {'t', 'z'},
 {'x'},
 {'x', 'z'},
 {'y'},
 {'x', 'y'},
 {'y', 'z'},
 {'x', 'y', 'z'},
 {'z'}]

## 4、示例：从新闻网站点击流中挖掘

In [56]:
parseDat = [line.split() for line in open('kosarak.dat').readlines()]
parseDat[:5]

[['1', '2', '3'], ['1'], ['4', '5', '6', '7'], ['1', '8'], ['9', '10']]

In [57]:
initSet = createInitSet(parseDat)
myFPtree, myHeaderTab = createTree(initSet, 100000)

In [58]:
myFreqList = []
mineTree(myFPtree, myHeaderTab, 100000, set([]), myFreqList)

  Null Set   1
   6   107404
  Null Set   1
   6   261773
  Null Set   1
   6   186289
    11   117401
   11   9718
  Null Set   1
   6   117401


In [59]:
len(myFreqList)

9

In [60]:
myFreqList

[{'1'},
 {'1', '6'},
 {'11'},
 {'11', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'6'}]

> 总结：FP-growth算法是一种用于发现数据集中频繁模式的有效方法，该算法利用Apriori原则，执行更快，数据集存储在称为FP树的结构中，通过查找元素项的条件基及构建条件FP树来发现频繁项集。